In [3]:
import requests

In [4]:
# request 唯一的作用是，模拟浏览器发送请求

## 爬取sogou首页的数据

In [6]:
# 指定URL
url = 'https://www.sogou.com/'

# 发请求
response = requests.get(url = url)

# 获取响应数据, 只有请求成功之后才能返回这个响应数据
page_text = response.text # 返回一个字符串

# 持久化存储
with open('./sogou.html', 'w', encoding = 'utf-8') as fp:
          fp.write(page_text)

print('数据爬取结束')

数据爬取结束


In [ ]:
# 但是这个网页丧失了样式，但是数据都还在

##  设计一个简单的网页采集器，搜狗爬取关键词结果页面

In [11]:
# 网页是'https://www.sogou.com/web?query=李振宁&_ast=1619735760&_asf=www.sogou.com&w=01029901&p=40040100&dp=1&cid=&s_from=result_up&sut=3110&sst0=1619735766394&lkt=0%2C0%2C0&sugsuv=00D219E62EC142EE6070667BE4E8E543&sugtime=1619735766394'
# 但是这个是静态的网页，而且如果只写到web?query=李振宁的话照样请求不了，所以需要一些处理
# 处理url携带的参数，多组参数

# 这里涉及到一个UA伪装，就是user_agents伪装，因为用requests来爬的话就是用requests的身份来爬，而加上UA后就是用你电脑的身份爬了
# UA伪装：门户网站会检测对应请求的身份标识，当门户网站发现是某一个浏览器，就说明该请求是正常的，一定不会拒绝
# 如果监测到请求不是基于浏览器，则表示该请求为不正常。
# 我目前的身份是：Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36

# UA伪装，将对应的user_agents封装到一个字典中（算头信息，所以用header）这里的header基于谷歌浏览器。
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

url = 'https://www.sogou.com/web'

# 将url携带的参数，封装到字典中，这样以后就可以直接输入然后去搜索了
kw = input('enter a word:')
params = {
    'query':kw
}

# 目前发起请求的是携带参数的，并且已经处理了(动态拼接)
# 进行UA伪装
requests.get(url = url, params = params, headers = headers)

page_text = response.text

fileName = '搜狗-' + kw + '.md'
with open(fileName, 'w', encoding = 'utf-8') as fp:
          fp.write(page_text)

print(fileName, '保存成功')

enter a word:lizh
搜狗-lizh.md 保存成功


##  破解百度翻译

In [44]:
# 使用百度翻译的时候，发现会局部刷新，url改变。所以需要用f12看一下ajax的请求。
# 每一个ajax对应的是每一个字符的翻译
# 需要按F12，然后找network的all右边的XHR，这个指ajax返回的信息，然后找到你请求的项目（逐个点击然后往下拉，看到自己最终想要翻译的句子）

# 对应的请求是POST请求，所以需要用requests做POST请求。
# 响应数据是json数据，是需要捕获的数据。
import requests

post_url = 'https://fanyi.baidu.com/sug' # 从f12的request url中获取

# UA伪装
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

# 这个data在f12最下方的Form Data中看到，也就是我们想翻译的浏览器上传的数据！
translate = input('input some words you want to translate to Chinese: ')
data = {
    'kw': translate
}

response = requests.post(url = post_url, data = data, headers = headers)


# 百度翻译响应回来的数据是一组json，因为从F12的Response Header中的Content-Type: application/json看出来！
# 用.json()直接返回的是obj（一定要之前确定服务器响应的是json类型的才行！）
dict_obj = response.json() # 这个返回的是翻译结果

# 存储
# fileName = './' + translate + '.json'
# fp = open(fileName, 'w', encoding = 'utf-8')

# json.dump()就是指对json文件进行编码！
# json.dump(dict_obj, fp = fp, ensure_ascii = False) # 因为有中文的地方不能用ASCII码！

# fp.close() # 这句话一定要加，不然显示大小为0，永远在打开的状态！


# 这里并不想存储，直接返回翻译结果：
print('\nThe meaning is: ')

# 防止返回的是空值
if dict_obj['data'] != []:
    print(dict_obj['data'][0]['v'])
else:
    print('\'',translate, '\'', 'Can\'t be translated!')

print('\n数据查询结束')



input some words you want to translate to Chinese: boring

The meaning is: 
adj. 无聊的，无趣的; 令人厌烦的; 单调的，乏味的 n. 钻孔; 钻屑 v. 令人厌烦(bor

数据查询结束


##  豆瓣电影分类排行榜

In [69]:
import requests

# 爬取豆瓣喜剧电影的排行榜的名字
# 打开网页，复制f12中的url
url = 'https://movie.douban.com/j/chart/top_list'

# 这个参数是从f12中的最下面的那个地方Query String Parameters找到的
param = {
    'type': '24', # 这里注意，所有的东西都要用string的形式！
    'interval_id': '100:90',
    'action': '', 
    'start': '0', # 这个大概意思就是从第0个开始取，再取10个
    'limit': '10'
}

# UA伪装
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

response = requests.get(url = url, params = param, headers = headers) # 这里用get是因为f12中显示的是get类型
list_data = response.json()

# 此时所有页面的信息都已经有了，现在把电影名列出来
movie_name = []
movie_rating = []
for i in range(len(list_data)):
    movie_name.append(list_data[i]['title'])
    movie_rating.append(float(list_data[i]['rating'][0]))

movie_info_dict = dict(zip(movie_name, movie_rating)) # 生成一个字典
movie_info_dict

{'憨豆先生精选辑': 9.6,
 '美丽人生': 9.5,
 '福尔摩斯二世': 9.5,
 '黄子华栋笃笑之金盆𠺘口': 9.5,
 '极品基老伴：完结篇': 9.4,
 '拽妹黛薇儿要上大学了没': 9.4,
 '放牛班的春天': 9.3,
 '触不可及': 9.3,
 '鬼子来了': 9.3,
 '摩登时代': 9.3}

##  肯德基地址爬取

In [77]:
# 找到肯德基含有地址信息的网站，是一个自己输入城市然后网站返回列表的一个做法
# 输入北京之后，点击查询，发现网址并没有变，所以这是一个ajax请求

import requests

url = 'http://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

# 输入地址（城市）
city = input('please input the city name in Chinese: ')
param = {
    'cname': '',
    'pid': '',
    'keyword': city,
    'pageIndex': '1', # this indicate the number of pages you want
    'pageSize': '10'
}

response = requests.post(url = url, params = param, headers = headers)

page_text = response.text

print(page_text)

{"Table":[{"rowcount":88}],"Table1":[{"rownum":1,"storeName":"育慧里","addressDetail":"小营东路3号北京凯基伦购物中心一层西侧","pro":"24小时,Wi-Fi,店内参观,礼品卡","provinceName":"北京市","cityName":"北京市"},{"rownum":2,"storeName":"京通新城","addressDetail":"朝阳路杨闸环岛西北京通苑30号楼一层南侧","pro":"24小时,Wi-Fi,店内参观,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":3,"storeName":"黄寺大街","addressDetail":"黄寺大街15号北京城乡黄寺商厦","pro":"Wi-Fi,点唱机,店内参观,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":4,"storeName":"四季青桥","addressDetail":"西四环北路117号北京欧尚超市F1、B1","pro":"Wi-Fi,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":5,"storeName":"亦庄","addressDetail":"北京经济开发区西环北路18号F1＋F2","pro":"24小时,Wi-Fi,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":6,"storeName":"石园南大街","addressDetail":"通顺路石园西区南侧北京顺义西单商场石园分店一层、二层部分","pro":"24小时,Wi-Fi,店内参观,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":7,"storeName":"北京南站","addressDetail":"北京南站候车大厅B岛201号","pro":"Wi-Fi,礼品卡","provinceName":"北京市","cityName":"北京市"},{"rownum":8,

In [101]:
# 但这里page_text是str格式，想要转换成字典，要用json包
import json

real_page_text = json.loads(page_text)

street_name = []
for i in range(len(real_page_text['Table1'])):
    street_name.append(real_page_text['Table1'][i]['storeName'])

street_name

['育慧里',
 '京通新城',
 '黄寺大街',
 '四季青桥',
 '亦庄',
 '石园南大街',
 '北京南站',
 '北清路',
 '大红门新世纪肯德基餐厅',
 '巴沟']

## 药监总局的爬取

In [124]:
# 化妆品生产许可证的数据
# http://scxk.nmpa.gov.cn:81/xk/ 新网址
# 这个cell只负责把所有企业的名字提取出来

import requests

url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}


all_page = 50 # 一份工375页，但是不知道为啥当超过50页的时候会出现数据异常，所以我们只取前50页的

all_names = []

# 现在json_data['list']里面就是我需要的所有公司的信息了
for i in range(all_page):

    param = {
        'on': 'true',
        'page': i,
        'pageSize': '15',
        'productName': '',
        'conditionType': '1',
        'applyname': '',
        'applysn': '',
    }

    response = requests.post(url = url, params = param, headers= headers)
    json_data = response.json()
    
    for k in range(len(json_data['list'])):
        all_names.append(json_data['list'][k]['EPS_NAME'])
    
    # 加一个进度条
    print(round(float(i/all_page), 4)*100, '% is done.')

all_names



0.0 % is done.
2.0 % is done.
4.0 % is done.
6.0 % is done.
8.0 % is done.
10.0 % is done.
12.0 % is done.
14.000000000000002 % is done.
16.0 % is done.
18.0 % is done.
20.0 % is done.
22.0 % is done.
24.0 % is done.
26.0 % is done.
28.000000000000004 % is done.
30.0 % is done.
32.0 % is done.
34.0 % is done.
36.0 % is done.
38.0 % is done.
40.0 % is done.
42.0 % is done.
44.0 % is done.
46.0 % is done.
48.0 % is done.
50.0 % is done.
52.0 % is done.
54.0 % is done.
56.00000000000001 % is done.
57.99999999999999 % is done.
60.0 % is done.
62.0 % is done.
64.0 % is done.
66.0 % is done.
68.0 % is done.
70.0 % is done.
72.0 % is done.
74.0 % is done.
76.0 % is done.
78.0 % is done.
80.0 % is done.
82.0 % is done.
84.0 % is done.
86.0 % is done.
88.0 % is done.
90.0 % is done.
92.0 % is done.
94.0 % is done.
96.0 % is done.
98.0 % is done.


['南京三盾药业有限公司',
 '青海环琼藏文化产品开发有限公司',
 '广东柏大化妆品有限公司',
 '南京佳原堂生物科技有限公司',
 '哈尔滨北星药业有限公司',
 '广州美颂化妆品有限公司',
 '惠州绵俪生物科技有限公司',
 '广东时尚女孩生物科技有限公司',
 '广州市合盛化妆品有限公司',
 '广东康王日化有限公司',
 '广州领衔生物科技有限公司',
 '安婕妤化妆品科技股份有限公司',
 '广东科盈科技有限公司',
 '汕头市薇漾生物科技有限公司',
 '广东名女人化妆品有限公司',
 '南京三盾药业有限公司',
 '青海环琼藏文化产品开发有限公司',
 '广东柏大化妆品有限公司',
 '南京佳原堂生物科技有限公司',
 '哈尔滨北星药业有限公司',
 '广州美颂化妆品有限公司',
 '惠州绵俪生物科技有限公司',
 '广东时尚女孩生物科技有限公司',
 '广州市合盛化妆品有限公司',
 '广东康王日化有限公司',
 '广州领衔生物科技有限公司',
 '安婕妤化妆品科技股份有限公司',
 '广东科盈科技有限公司',
 '汕头市薇漾生物科技有限公司',
 '广东名女人化妆品有限公司',
 '广州高迪生物科技有限公司',
 '富甲生物科技（广州）有限公司',
 '深圳市吉爱丝迪生物科技有限公司',
 '广州美珍生物科技有限公司',
 '广州至美化妆品有限公司',
 '汕头市潮南区彩芙雅化妆品有限公司',
 '广州欧滋化妆品有限公司',
 '广州御艾化妆品有限公司',
 '广州市佳贤精细化工有限公司',
 '广州科盈化妆品有限公司',
 '明辉实业（深圳）有限公司',
 '广州洁美化妆品有限公司',
 '广州娜艾施化妆品有限公司',
 '广州新济薇娜生物科技有限公司',
 '广州市科臣生物技术有限公司',
 '广州荃美生物科技有限公司',
 '广州市贝嘉欣化妆品有限公司',
 '广州市品华日化科技有限公司',
 '广州市花木采研生物科技有限公司',
 '广州市柏乐丝化妆品有限公司',
 '广州市大研生物技术有限公司',
 '广州凡岛化妆品有限公司',
 '广州五羊日化有限公司',
 '广州市盛龙口腔清洁用品有限公司',
 '广州微肽生物科技有限公司',
 '重庆灵方生物技术有限公司',
 '昌吉市爱互玛化妆品有限责任公司',
 '海南京润珍珠生物技术股

In [127]:
# 现在找到方法把每一家企业的详情给提取出来

import requests

url = 'http://scxk.nmpa.gov.cn:81/xk/#'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

param = {
    'on': 'true',
    'page': 1,
    'pageSize': '15',
    'productName': '',
    'conditionType': '1',
    'applyname': '',
    'applysn': '',
}

response = requests.get(url = url, headers = headers).text
# response

# 这里发现，如果你把这个东西转换成html然后与原网页对比，然后就会发现这里面不存在任何数据，但是原网页有，这就说明网页中的
# 数据是动态的，也就是说，想用药监总局的官网地址是请求不到数据的，需要用ajax才能提取到。

In [135]:
# 点开每一家公司的界面，看f12，发现所有数据都是动态的，param里面就是ID，所以直接摘取url，然后把ID放到param里，就OK了

import requests

url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

param = {
    'on': 'true',
    'page': '1',
    'pageSize': '15',
    'productName': '',
    'conditionType': '1',
    'applyname': '',
    'applysn': '',
}

response = requests.post(url = url, params = param, headers= headers)
json_data = response.json()

# 对每一家企业来说，url是（当发现网页是动态数据的时候得看ajax里面的网址了）
url_new = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsById'

# 输出前10个公司法人名字
legal_person = []

for i in range(10): 

    id_company = {
        'id': json_data['list'][i]['ID']
    }

    # 我们现在取新的每个公司的信息
    response_company_page = requests.post(url = url_new, headers= headers, params = id_company)

    data_individual = response_company_page.json()
    
    legal_person.append(data_individual['legalPerson'])

legal_person

['陈英', '万德才郎', '贺学兵', '冯全服', '张立国', '梁嘉宏', '张汉云', '陈维深', '何世文', '柯训忠']